# Distilling knowledge in models pretrained on CIFAR-10/100 datasets, using ***torchdistill***

## 1. Make sure you have access to GPU/TPU
Google Colab: *Runtime* -> *Change runtime type* -> *Hardware accelarator*: "GPU" or "TPU"

In [1]:
!nvidia-smi

Mon Feb  5 12:57:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 2. Install ***torchdistill***

In [2]:
!pip install torchdistill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.5 MB/s eta 0:00:00


## 3. Clone ***torchdistill*** repository to use its example code and configuration files

In [3]:
!git clone https://github.com/Moon-kimchi/torchdistill.git

Cloning into 'torchdistill'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 151 (delta 14), reused 0 (delta 0), pack-reused 126
Receiving objects: 100% (151/151), 319.69 KiB | 15.22 MiB/s, done.
Resolving deltas: 100% (35/35), done.


## 4. Distill knowledge in models pretrained on CIFAR-10

Note that the hyperparameters of ResNet were chosen based on either train/val (splitting 50k samples into train:val = 45k:5k) or cross-validation, according to the original papers.  
For the final run (once the hyperparameters are finalized), the authors used all the training images (50k samples).  
- ResNet: https://github.com/facebookarchive/fb.resnet.torch

The following examples demonstrate how to 1) tune hyperparameter and 2) do final-run with ResNet-20 on CIFAR-10 dataset, respectively.

### 4.1 Hyperparameter tuning based on train:val = 45k:5k
Let's start with a small **student model**, ResNet-20, with a pretrained DenseNet-BC (k=12, depth=100) as a **teacher model** for tutorial.  

Open `torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml` and update hyperparameters as you wish e.g., number of epochs (*num_epochs*), batch size (*batch_size* in *train_data_loader* entry), learning rate (*lr* within *optimizer* entry), and so on.
By default, the hyperparameters in the example config are identical to those in the final run config.
  
You will find a lot of module names from [PyTorch documentation](https://pytorch.org/docs/stable/index.html) and [torchvision](https://pytorch.org/docs/stable/torchvision/) such as [`SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD), [`MultiStepLR`](https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.MultiStepLR), [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss), [`CIFAR10`](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.CIFAR10), [`RandomCrop`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.RandomCrop) (, and more). You can update their parameters or replace such modules with other modules in the packages. For instance, `SGD` could be replaced with [`Adam`](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam), and then you will change the parameters under `params` (at least delete `momentum` entry as the parameter is not for `Adam`).

In [4]:
!python torchdistill/examples/torchvision/image_classification.py --config torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml --run_log log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.log

2024/02/05 12:57:56	INFO	torchdistill.common.main_util	Not using distributed mode
2024/02/05 12:57:56	INFO	__main__	Namespace(config='torchdistill/configs/sample/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.yaml', device='cuda', run_log='log/cifar10/kd/resnet20_from_densenet_bc_k12_depth100-hyperparameter_tuning.log', start_epoch=0, seed=None, disable_cudnn_benchmark=False, test_only=False, student_only=False, log_config=False, world_size=1, dist_url='env://', adjust_lr=False)
2024/02/05 12:57:56	INFO	torchdistill.common.main_util	Getting `CIFAR10` from `torchvision.datasets`
2024/02/05 12:57:56	INFO	torchdistill.common.main_util	Calling `CIFAR10` from `torchvision.datasets` with {'kwargs': {'root': '~/datasets/cifar10', 'train': True, 'download': True}}
100% 170498071/170498071 [00:13<00:00, 13027330.94it/s]
Extracting /root/datasets/cifar10/cifar-10-python.tar.gz to /root/datasets/cifar10
2024/02/05 12:58:13	INFO	torchdistill.common.main_util	Getting `Rando

At the end of the training process, you will see improved accuracy of the student model (ResNet-20) compared to that trained without teacher in another example notebook and/or the accuracy reported in [the ResNet paper (Table 6)](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf).